In [ ]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv

start = datetime.datetime.now()

# 搜尋條件
city = "墨爾本" 
checkin_time = "2021-05-20" 
checkout_time = "2021-05-21"

# 網址處理
url = "https://tw.hotels.com/"

# 擷取網頁
# ！！！記得改driver路徑！！！
driverPath = 'C:\Shared\BDSE_網路爬蟲\chromedriver.exe'
browser = webdriver.Chrome(executable_path=driverPath)
browser.implicitly_wait(5)                                  # 等待網頁載入
browser.get(url)                                            # 網頁下載至瀏覽器  
time.sleep(5)
searchNode = browser.find_elements_by_xpath("//input[contains(@id,'qf-0q-destination')]")
checkInNode = browser.find_elements_by_xpath("//input[contains(@id,'qf-0q-localised-check-in')]")
checkOutNode = browser.find_elements_by_xpath("//input[contains(@id,'qf-0q-localised-check-out')]")

# 輸入搜尋關鍵資料
searchNode[0].clear()                                       # 清除城市欄位
searchNode[0].send_keys(city)                               # 輸入搜尋城市
time.sleep(2)
searchNode[0].send_keys(Keys.TAB)                           # 按Tab跳到check in欄位
checkInNode[0].clear()                                      # 清除check in欄位                                          
checkInNode[0].send_keys(checkin_time)                      # 輸入check in時間
time.sleep(2)
searchNode[0].send_keys(Keys.TAB)                           # 按Tab跳至check out欄位
checkOutNode[0].clear()                                     # 清除check out欄位
checkOutNode[0].send_keys(checkout_time)                    # 輸入check out時間
time.sleep(2)
checkOutNode[0].send_keys(Keys.ENTER)                       # 表單按Enter表示輸入完成        
time.sleep(2)


def scroll():
    innerHeightOfWindow = 0    #視窗高度起始值=0
    totalOffset = 0            #滾動距離起始值=0

    # 無限迴圈，儘量不要這麼寫，要確保有break終止程式的設計！
    while True:
        
        #偵測視窗高度
        innerHeightOfWindow = browser.execute_script('return window.document.documentElement.scrollHeight;');

        #設定滾動距離等於偵測高度
        totalOffset = innerHeightOfWindow-1000;       
        
        js_scroll = '''(
            function (){{
                window.scrollTo({{
                    top:{}, 
                    behavior: 'instant' 
                }});
            }})();'''.format(totalOffset)
        #滾動
        browser.execute_script(js_scroll)
        
        time.sleep(0.5)
        
        # 顯示目前滾動的距離
        if innerHeightOfWindow-totalOffset>2000:
            print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        #導入Soup解析最底會出現的：沒有空房的文字，如果文字出現，終止跳脫迴圈
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        end = soup.find(id = 'result-info-container')
        if  len(end)>0:
            break

# 執行scroll這個func        
scroll()

# 找出所有飯店的框框
hotels_renderers = browser.find_elements(By.CSS_SELECTOR, 'section.hotel-wrap')
        
# 開csv檔，以空白分隔欄位，建立 CSV 檔寫入器
# 檔名是城市_今天日期.csv
with open(f'{city}_{datetime.date.today()}.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # 寫入一列資料
    writer.writerow(['旅館名稱', '旅館地址', '旅館星級','旅館定價'])

    # 逐一檢視元素
    for element in hotels_renderers:
        # 印出分隔文字
        print("=" * 30)
        
        # 旅館名稱
        hotelPath = element.find_element(By.CSS_SELECTOR, "a.property-name-link")
        name = hotelPath.text
        print(name)
        
        # 旅館地址
        addressPath = element.find_element(By.CSS_SELECTOR, "span.address")
        address = addressPath.text
        print(address)

        # 旅館星級
        try:
            # 正常有星級
            starPath = element.find_element(By.CSS_SELECTOR, "span.star-rating-text")
            star = starPath.text
        except:
            # 無顯示
            star = "No Rating"
        print(star)

        # 旅館定價
        try:
            # 正常價格
            listingpricePath = element.find_element(By.CSS_SELECTOR, "div.price > a > strong")
        except:
            # 特價
            listingpricePath = element.find_element(By.CSS_SELECTOR, "div.price > a > ins")
        lprices = listingpricePath.text
        print(lprices)
        
    #    寫入CSV檔案中
        writer.writerow([name, address, star, lprices])